# Model practability

We now loook at at the practability of the previous presented MSPP-PD models.  

To do so we consider different combination of increasing larger network sizes ad network congestion and then we examine the average time that each problem model takes to be solved.  

In [1]:
import gurobipy as gb
from gurobipy import GRB
import pandas as pd
import numpy as np

from utils import data_readers as dr
from utils import problem_models as pm

## Read datas

We start by loading the network instances:  
- For each network topology (6x6, 8x8, 10x10, 12x12) we load the dataframe that contains all the instaces for that type of network;
- And then we append each dataframe to a list.

In [20]:
network_types = ["6x6", "8x8", "10x10", "12x12"]
idx_of_network_type = dict(zip(network_types, range(len(network_types))))
network_dfs = []

for network_type in network_types:
    network_filename = f"data/d_it_ij_{network_type}_10it.csv"
    network_dfs.append(dr.read_networks_csv(network_filename,
                                            along="cols"))

network_type_to_its_df = dict(zip(network_types, network_dfs))

# an example
network_dfs[0].head()

0                   1                             2             \
           6         7         6         7         8         7         8    
it1  1.612014  1.281732  1.512410  1.612053  0.495080  1.633618  0.282487   
it2  0.345666  1.549119  1.935411  1.664691  1.150243  1.346757  0.790995   
it3  1.059043  0.470901  0.397252  0.210863  0.720249  0.926994  1.987635   
it4  0.297746  1.561490  0.480028  0.058043  1.032561  1.155090  0.997398   
it5  1.296527  0.536009  0.439805  0.458544  1.420302  0.178641  0.108120   

                     3             ...        26                  27  \
           9         8         9   ...        32        33        32   
it1  1.570645  0.398975  0.890572  ...  0.491645  1.860557  0.792445   
it2  0.558598  1.677854  0.874843  ...  1.001069  0.476828  0.640806   
it3  0.360284  0.508765  1.101995  ...  1.213463  1.572754  1.330204   
it4  0.577482  1.140927  0.833327  ...  0.279092  1.068302  1.286386   
it5  0.772551  1.102323  0.592525  ...  0.652133  0.808668  1.197061   

                               28                            29            
           33        34        33        34        35        34        35  
it1  0.644937  0.296105  1.470566  1.336894  1.595893  0.273600  1.416396  
it2  1.368508  0.503312  0.577109  1.779668  1.729859  0.117643  0.830738  
it3  0.851481  1.894534  1.456438  1.425572  1.092434  1.438385  0.934447  
it4  0.255251  0.459456  1.479642  1.490551  0.335991  1.382244  1.578896  
it5  0.794697  0.263786  1.622354  1.549742  1.855342  0.732395  0.670513  

[5 rows x 80 columns]

## Manage datas

Here we define other variables that will be used in the practability analysis 

In [3]:
num_istances = 5

# problem_types = ["MSPP","ABP", "NBP", "ALP", "NLP", "AQP", "NQP"]
problem_types = ["MSPP", "ABP"]
idx_of_pb_type = dict(zip(problem_types, range(len(problem_types))))


# congestion_levels = [0.5, 1, 1.5, 2]
congestion_levels = [0.5, 1]
idx_of_congestion_lvl = dict(zip(congestion_levels, range(len(congestion_levels))))


time_limit = 10 * 60  # (s)
convergence_times_shape = (len(network_types),  # each network type
                           num_istances,       # has a number of different instances 
                           len(problem_types), # and we'll solve ...
                           len(congestion_levels)
                           )
convergence_times = np.full(convergence_times_shape, fill_value=np.nan)

# TODO: consider num_of_networks that does not converge

## Solve each case

In [ ]:
# Skip this cell if you want to use pre-computed results

for network_type in network_types:

     networks_df = network_type_to_its_df[network_type]
     networks_shape = [int(i) for i in network_type.split("x")]
     network_nodes = pm.get_nodes(networks_df)

     for it_i, network_arcs in enumerate(pm.network_instances(networks_df.head(num_istances))):

          for pb_type in problem_types:
               
               for congestion_lvl in congestion_levels:

                    agents = pm.generate_agents(networks_shape, congestion_lvl)

                    pb, X, *_ = pm.set_problem(pb_type, network_nodes, network_arcs, agents)
                    pb.Params.TimeLimit = time_limit
                    pb.optimize()

                    if pb.status == GRB.OPTIMAL:
                         convergence_times[idx_of_network_type[network_type],
                                           it_i,
                                           idx_of_pb_type[pb_type],
                                           idx_of_congestion_lvl[congestion_lvl]] = pb.runtime
                    elif pb.status == GRB.TIME_LIMIT:
                         print(f"A {pb_type} problem exceeded the time limit on a {network_type} network with {len(agents)} agents.")
                    else:
                         # TODO: print some error or throw exception
                         pass

If desired we can store results in order to avoid waiting for computations each time we run the notebook

In [24]:
np.save("results/models_convergence_times.npy", convergence_times)

## Final statistic

In the following we report the average convergence time (s) for MSPP-PD model variants' for different network sizes and congestion levels.

Load the results obtained from previous chapter 

In [26]:
convergence_times = np.load("results/models_convergence_times.npy")

And print the results (along rows we have the variant of the model and along columns we have the different congestion levels) 

In [27]:
avg_convergence_times = np.nanmean(convergence_times, axis=1)

for network_type in network_types:
    network_avg_convergence_time = avg_convergence_times[idx_of_network_type[network_type], :, :]
    network_avg_convergence_time_df = pd.DataFrame(network_avg_convergence_time,
                                                   columns=congestion_levels,
                                                   index=problem_types)
    print(f"Average convercenge time for {network_type} networks:")
    print(network_avg_convergence_time_df)
    print()

Average convercenge time for 6x6 networks:
           0.5       1.0
MSPP  0.001487  0.003064
ABP   0.005520  0.013042

Average convercenge time for 8x8 networks:
           0.5       1.0
MSPP  0.003748  0.006777
ABP   0.014749  0.052971

Average convercenge time for 10x10 networks:
           0.5       1.0
MSPP  0.007403  0.013063
ABP   0.080461  0.645724

Average convercenge time for 12x12 networks:
           0.5       1.0
MSPP  0.014038  0.027263
ABP   0.210606  6.089673

